In [ ]:
##########################################

# Start here once given anonymized csvs.

###########################################
# %matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import os
import LLB_custom_scripts_mac
from LLB_custom_scripts_mac import make_module_summary
import csv
from collections import Counter
import re

In [ ]:
# Loop through anonymized files and save the dataframes 
# of progression and module files in separate dictionaries.
# Also make a dictionary with the number of assessments in 
# each module, each year.

# Paths for locating anonymized csvs:
anon_module_csv_path = ("/Users/Kate/Desktop/Vicky project/"
                        "Data/anonymized module csvs/")

anon_progression_csv_path = ("/Users/Kate/Desktop/Vicky project/"
                            "Data/anonymized progression csvs/")

progression_files = {}
module_files = {}
n_assessments_module_dict = {}

# Loop
for filename in os.listdir(anon_module_csv_path):
    if filename == '.DS_Store': 
        continue
    else:
        # Convert csv to dataframe and store in a dictionary.
        module_name = f'{filename}'.split('.')[0]
        module_files[module_name] = pd.read_csv(
            anon_module_csv_path + f'{filename}')
        module_files[module_name].set_index('SPR Code', inplace=True)
        num_assessments = module_files[module_name].columns[-1].split(' ')[1]
        n_assessments_module_dict[module_name] = num_assessments
for filename in os.listdir(anon_progression_csv_path):
    if filename == '.DS_Store': 
        continue
    else:
        # Convert csv to dataframe and store in a dictionary.
        progression_files[f'{filename}'.split('.')[0]] = pd.read_csv(
            anon_progression_csv_path + f'{filename}')
        progression_files[f'{filename}'.split('.')[0]].set_index(
            'SPR Code', inplace=True)

In [ ]:
# Store the 201011 module files for comparison
anon_module_path_200910 = ("/Users/Kate/Desktop/Vicky project/"
                        "Data/anonymized module csvs - starting 2009-2010/")

mod_files_2010 ={}
for filename in os.listdir(anon_module_path_200910):
    if filename == '.DS_Store':
        continue
    else:
        year, module, tmp = f'{filename}'.split('.')[0].split('_')
        if year == '201011':
            df = pd.read_csv(anon_module_path_200910 + f'{filename}')
            mod_files_2010[module] = df.set_index('SPR Code')
                                                 

In [ ]:
# Remove all students with blank results
for dfname, df in module_files.items():
    module_files[dfname] = df[df['Result'].notnull()]

In [ ]:
# Remove all students not in the LLB programme
for dfname, df in module_files.items():
    module_files[dfname] = df[df['Programme'].str.startswith('LLB Law')] 

In [ ]:
# Remove all duplicated student entries from module dataframes 
# since confusion about a variable, will have to remove all that were duplicated
for dfname, df in module_files.items():
    module_files[dfname] = df[~df.index.duplicated(keep=False)]

In [ ]:
# Add reassess flag
def determine_reassess_and_dr_flags(df):
    df['Reassess Flag'] = df.apply(lambda row: any(row.isin(['FR'])), axis = 1)   
    df['DR Flag'] = df.apply(lambda row: (any(row.isin(['DR'])) & (row['Result'] != 'D')), axis = 1) 
    ######### better way to catch assessment DRs only??? ########
    return df
            
for dfname, df in module_files.items():
    module_files[dfname] = determine_reassess_and_dr_flags(df)

In [ ]:
# Create student records dictionary

# create dataframes with 
student_attempts = pd.DataFrame()
student_grades = pd.DataFrame()
student_reassess_flags = pd.DataFrame()
student_dr_flags = pd.DataFrame()
student_marks = pd.DataFrame()

for dfname, df in module_files.items():
    year, module, tmp = dfname.split('_')
    student_attempts = student_attempts.join(df[['Attempt']], how='outer')
    student_attempts.rename(columns={'Attempt': f'{year} {module}'}, inplace=True)
    student_grades = student_grades.join(df[['Grade']], how='outer')
    student_grades.rename(columns={'Grade': f'{year} {module}'},inplace=True)
    if any(df.columns == 'Mark'):
        student_marks = student_marks.join(df[['Mark']], how='outer')
        student_marks.rename(columns={'Mark':f'{year} {module}'}, inplace=True)
    else:
        student_marks = student_marks.join(df[['Grade']], how='outer')
        student_marks.rename(columns={'Grade':f'{year} {module}'}, inplace=True)
        student_marks[f'{year} {module}'] = np.NaN
    student_reassess_flags = student_reassess_flags.join(df[['Reassess Flag']], how='outer')
    student_reassess_flags.rename(columns={'Reassess Flag': f'{year} {module}'},inplace=True)
    student_dr_flags = student_dr_flags.join(df[['DR Flag']], how='outer')
    student_dr_flags.rename(columns={'DR Flag': f'{year} {module}'},inplace=True)
    
student_attempts = student_attempts.reindex(sorted(student_attempts.columns), axis=1)#sort by year

In [ ]:
student_zipped = pd.DataFrame(index = student_attempts.index, columns = student_attempts.columns)
for index in student_attempts.index:
    for column in student_attempts.columns:
        if student_marks.at[index, column] == np.NaN:
            student_zipped.at[index, column] = str(student_reassess_flags.at[index, column]) \
                                            + ' ' + str(student_dr_flags.at[index, column]) \
                                            + ' ' + str(student_attempts.at[index,column]) \
                                            + ' ' + 'nan' + ' ' + str(student_grades.at[index,column])
        else:
            student_zipped.at[index, column] = str(student_reassess_flags.at[index, column]) \
                                            + ' ' + str(student_dr_flags.at[index, column]) \
                                            + ' ' + str(student_attempts.at[index,column]) \
                                            + ' ' + str(student_marks.at[index,column]) \
                                            + ' ' + str(student_grades.at[index,column])

In [ ]:
student_zipped.head()

In [ ]:
student_zipped = student_zipped.replace('nan nan nan nan nan', np.NaN)

records = {}

for index, row in student_zipped.iterrows():
    courses_on_record = row.dropna().index.tolist()
    results = row.dropna().values.tolist()
    records[index] = list(zip(courses_on_record,results))

RECORDS = {}
for SPRcode, record in records.items():
    new_list = []
    for (course, result) in record:
        new_list.append(course + ' ' + result)
    RECORDS[SPRcode] = new_list
    
len(RECORDS)

In [ ]:
student_zipped.head()

In [ ]:
RECORDS

In [ ]:
# make dictionaries about student variables for inputting into a future 'STUDENTS' table (clean this up too)
year_entered = {}
graduated_year = {}
status_now = {}
STUDENTS = pd.DataFrame(columns=['Entire Record',
                                 'CONAD History','CONTRACT History', 'LSM History', 'TORT History', 
                                 'LAND History', 'CRIMINAL History','EQUITY History', 'EU History', 
                                 'LT1 History', 'LT2 History', 'CONAD Attempts', 'CONTRACT Attempts',
                                'LSM Attempts', 'TORT Attempts', 'LAND Attempts', 'CRIMINAL Attempts',
                                'EQUITY Attempts', 'EU Attempts', 'LT1 Attempts', 'LT2 Attempts'])

list_years = ['201112','201213','201314','201415','201516', '201617','201718']
for SPRcode, list_records in RECORDS.items(): # for all students...
    try:
        first_year = list_records[0].split()[0]
        last_year = list_records[-1].split()[0]
        # fill in entire record column
        STUDENTS.at[SPRcode, 'Entire Record'] = list_records
        # if left prematurely -> does not include any students that left after 2018
        if not any([(record.split()[0] == '201718') for record in list_records]):
            if not any([record.split()[1] == 'LT2' for record in list_records]):     
                left_prematurely_year_courses = []
                for record in list_records:
                    if record.split()[0] == last_year:
                        left_prematurely_year_courses.append((record.split()[1] + ' ' + record.split()[2]
                                                              + ' ' + record.split()[3] + ' ' + record.split()[4]
                                                              + ' ' + record.split()[5] + ' ' + record.split()[6]))
                # record last year and courses taken that year
#                 year_left_prematurely[SPRcode] = last_year
#                 courses_prior_to_leaving_prematurely[SPRcode] = left_prematurely_year_courses
                status_now[SPRcode] = 'Left prematurely'
        # if graduauted
        if any([((record.split()[1] == 'LT2') & (record.split()[-1] == 'P')) for record in list_records]):
            if any([((record.split()[1] == 'LT1') & (record.split()[-1] == 'P')) for record in list_records]):
                if any([((record.split()[1] == 'EU') & (record.split()[-1] == 'P')) for record in list_records]):
                    graduated_year[SPRcode] = last_year
                    status_now[SPRcode] = 'Graduated'
        # determine current status (progressing or not progressing) if not graduated or left
        if SPRcode not in status_now.keys():
            # determine if all courses passed for 201718
            if any([((record.split()[0] == '201718') & (record.split()[-1] == 'F')) for record in list_records]):
                status_now[SPRcode] = 'Not progressing'
            elif any([(record.split()[0] =='201718') for record in list_records]):
                status_now[SPRcode] = 'Progressing'
        # determine year entered if possible
        for year in list_years[1:]:
            if first_year == year:
                if any([f'{year} CONAD' in record for record in list_records]):
                    if any([f'{year} CONTRACT' in record for record in list_records]):
                        year_entered[SPRcode] = year
        if (first_year == '201112') & (SPRcode not in mod_files_2010[module].index):
            if any([f'{first_year} CONAD' in record for record in list_records]):
                if any([f'{first_year} CONTRACT' in record for record in list_records]):
                    year_entered[SPRcode] = first_year          
    except:
        print(SPRcode, list_records)
        
# find the PT/FT status of each student (clean this up or make function above)
student_pt_ft_statuses = pd.DataFrame()

for dfname, df in module_files.items():
    year, module, tmp = dfname.split('_')
    student_pt_ft_statuses = (student_pt_ft_statuses.join(df[['Programme']], how='outer'))
    student_pt_ft_statuses.rename(columns={'Programme': f'{year} {module}'}, inplace=True)

student_pt_ft_statuses = student_pt_ft_statuses.reindex(sorted(student_pt_ft_statuses.columns), axis=1)#sorted by year

records = {}
for index, row in student_pt_ft_statuses.iterrows():
    courses_on_record = row.dropna().index.tolist()
    programmes_listed = row.dropna().values.tolist()
    records[index] = list(zip(courses_on_record,programmes_listed))
    
changed_programme = {}
programmes = {}
for SPRcode, record in records.items():
    first_programme = record[0][1]
    for (course, programme) in record:
        if programme != first_programme:
            changed_programme[SPRcode] = record
    if SPRcode in changed_programme:
        programmes[SPRcode] = 'Changed'
    else:
        if 'part' in first_programme:
            programmes[SPRcode] = 'PT'
        elif 'full' in first_programme:
            programmes[SPRcode] = 'FT'  
        elif '6' in first_programme:
            programmes[SPRcode] = '6YR'

In [ ]:
# Make STUDENTS dataframe
STUDENTS['Programme'] = pd.Series(programmes)
STUDENTS['Year started'] = pd.Series(year_entered)
STUDENTS['Current status'] = pd.Series(status_now)
STUDENTS['Graduated year'] = pd.Series(graduated_year)
# STUDENTS['Year left prematurely'] = pd.Series(year_left_prematurely)
# STUDENTS['Courses prior to leaving prematurely'] = pd.Series(courses_prior_to_leaving_prematurely)
cols = ['Programme','Year started', 'Current status', 'Graduated year', 'Entire Record', 
        'LSM History', 'CONTRACT History', 'CONAD History','TORT History',  'CRIMINAL History', 'LAND History',
       'EQUITY History', 'EU History', 'LT1 History', 'LT2 History', 'LSM Attempts', 'CONTRACT Attempts',
                                 'CONAD Attempts', 'TORT Attempts', 'CRIMINAL Attempts', 'LAND Attempts',
                                'EQUITY Attempts', 'EU Attempts', 'LT1 Attempts', 'LT2 Attempts']
STUDENTS = STUDENTS[cols]

In [ ]:
# check that all 
any(STUDENTS['Entire Record'].isnull())

In [ ]:
# determine the last time that the course shows up, and the attempt number of that time 
# also could count the number of attempts myself
for name in ['CONAD History','CONTRACT History', 'LSM History', 'TORT History', 
            'LAND History', 'CRIMINAL History','EQUITY History', 'EU History', 
            'LT1 History', 'LT2 History']:
    STUDENTS[name] = np.empty((len(STUDENTS),0)).tolist()

def record_history(row):
    for record in row['Entire Record']:
        print(record)
        year, course, reassess_flag, dr_flag, attemptnum, mark, result = record.split()
        row[f'{course} History'].append(record)

STUDENTS[STUDENTS['Year started'].notnull()].apply(record_history, axis=1);

In [ ]:
# this is what I need to fix -> how to make parse dr and fr attempts properly 
# and also catch where the administration didn't 

for name in ['LSM Attempts','CONTRACT Attempts','CONAD Attempts', 'TORT Attempts', 
        'CRIMINAL Attempts','LAND Attempts', 'EQUITY Attempts', 'EU Attempts', 
        'LT1 Attempts', 'LT2 Attempts']:
    STUDENTS[name] = np.empty((len(STUDENTS),0)).tolist()

for course in ['CONAD','CONTRACT','LSM','TORT','EQUITY','LAND','CRIMINAL','EU','LT1','LT2']:
    for index, row in STUDENTS.iterrows():
        for record in row[f'{course} History']:
            year, course, reassess_flag, dr_flag, attemptnum, mark, result = record.split()
            next_year = year.split('_')[0][:2] + \
                        str(int(year.split('_')[0][2:4]) + 1) + \
                        str(int(year.split('_')[0][4:]) + 1)
            previous_year = year.split('_')[0][:2] + \
                        str(int(year.split('_')[0][2:4]) - 1) + \
                        str(int(year.split('_')[0][4:]) - 1)
            if index not in mod_files_2010[module].index: 
                # only calculating the attempt nums 
                # for students that entered 2011 or later
                attemptnum = int(float(attemptnum)) #2
                if any(row[f'{course} Attempts']): # any attempts already for this course and student id
                    if type(row[f'{course} Attempts'][-1][-1]) == list:  # if there were multiple previous attempts
                        most_recent_previous_attempt = float(row[f'{course} Attempts'][-1][-1][-1])
                    else: # if there was a single previous attempt   
                        most_recent_previous_attempt = float(row[f'{course} Attempts'][-1][-1])
                    attempt_diff = int(float(attemptnum) - most_recent_previous_attempt)
                        #subtract the most recent previous attempt
                    if attempt_diff == 0:
                        previous_year_name = previous_year + '_' + module + '_' + 'assessment'
                        try:
                            previous_year_result = module_files[previous_year_name].at[index,'Result']
                        except:
                            previous_year_result = np.NaN
                        if previous_year_result == 'D':
                            if attemptnum == 1:
                                attemptnum = int(most_recent_previous_attempt)
                            elif (reassess_flag == 'True'): 
                                # could add 2 or 3 attempts here, depending whether there is a third assessment the following year
                                attemptnum = int(most_recent_previous_attempt) + attemptnum
                        else: # catch where attemptnum doesn't increment
                            attemptnum = int(most_recent_previous_attempt) + attemptnum
                else: # if no previous attempts recorded
                    attempt_diff = int(float(attemptnum)) 
                if dr_flag == 'True': # student supposed to have had one official attempt that year, but sometimes this doesn't happen 
                    if (attempt_diff == 3): # admin counted DR as an attempt
                        row[f'{course} Attempts'].append((year, ['' ,''], [attemptnum-2, attemptnum-1]))
                        row[f'{course} Attempts'].append((next_year, result, attemptnum))
                    elif (attempt_diff == 2):
                        row[f'{course} Attempts'].append((year, ['', result], [attemptnum-1, attemptnum]))
                    else:
                        #the DR was not counted by admin but had an effect that should be recorded
                        row[f'{course} Attempts'].append((year, ['', result], [attemptnum, attemptnum]))
                elif (reassess_flag == 'True'): # if attempt that year included an FR
                    if (attempt_diff == 3): # if the difference between the previous attempt and this one is 3
                        # there is an attempt and a reassessment that year
                        # and another reassessment the next year that isn't recorded there      
                        row[f'{course} Attempts'].append((year, ['', ''], [attemptnum-2, attemptnum-1]))
                        row[f'{course} Attempts'].append((next_year, result, attemptnum))
                    elif (attempt_diff == 2):
                        row[f'{course} Attempts'].append((year, ['', result], [attemptnum-1, attemptnum]))
                    elif (attempt_diff == 1):
                        print('attempt_diff error', index, record, '*', row[f'{course} History'])
                        row[f'{course} Attempts'].append((year, result, attemptnum+1))
                else:
                    row[f'{course} Attempts'].append((year, result, attemptnum))
            else:
                row[f'{course} Attempts'] = []

In [ ]:
year, result, attemptnum

In [ ]:
# STUDENTS.drop(labels=['Year 1 Courses', 'Year 2 Courses', 
#         'Year 3 Courses', 'Year 4 Courses', 'Year 5 Courses', 'Year 6 Courses'], axis=1, inplace=True)

In [ ]:
STUDENTS.to_excel("/Users/Kate/Desktop/Vicky project/data/STUDENTS.xls")

In [ ]:
STUDENTS['Programme'].value_counts()

In [ ]:
# Calculate metrics and create visualizations

# what proportion of students graduated within 6 years (PT) or 5 years (FT) for students starting in 2011, 2012, 2013?
# write function or loop to analyze each programme by year entered (2 by 3)

years_entered = pd.Index(['2011', '2012'])
programmes = pd.Index(['FT', 'PT'])

graduation_rate_df = pd.DataFrame(index=programmes, columns=years_entered) 

for programme in programmes.tolist():
    for year in years_entered.tolist():
        if programme == 'PT':
            cutoff = int(year) + 6
        elif programme == 'FT':
            cutoff = int(year) + 5
        selection_total_students = STUDENTS['Year started'].str.contains(year).fillna(False) \
                                    & (STUDENTS['Programme'] == programme)
        selection_graduated_students = (selection_total_students 
                                        & (STUDENTS.loc[STUDENTS['Graduated year'].notnull(), 'Graduated year']
                                           .apply(lambda x: x[:2] + x[-2:])
                                           .astype('int') 
                                           <= cutoff))
        graduation_rate_df.at[programme, year] = (len(STUDENTS[selection_graduated_students]) 
                                                  / len(STUDENTS[selection_total_students]))
graduation_rate_df

In [ ]:
# what proportion of students graduated within 6 years (PT) or 5 years (FT) for students starting in 2011, 2012, 2013?
# write function or loop to analyze each programme by year entered (2 by 3)

###############################################################################
'''
this isn't quite right, because the 3rd attempts are actually the following year
'''
###############################################################################

print('Leaving early rate: ')
years_entered = pd.Index(['2011', '2012'])
programmes = pd.Index(['FT', 'PT'])

leaving_rate_df = pd.DataFrame(index=programmes, columns=years_entered)

for programme in programmes.tolist():
    for year in years_entered.tolist():
        if programme == 'PT':
            cutoff = int(year) + 6
        elif programme == 'FT':
            cutoff = int(year) + 5
        selection_total_students = STUDENTS['Year started'].str.contains(year).fillna(False) \
                                    & (STUDENTS['Programme'] == programme)
        selection_left_students = (selection_total_students 
                                        & (STUDENTS.loc[STUDENTS['Year left prematurely'].notnull(), 'Year left prematurely']
                                           .apply(lambda x: x[:2] + x[-2:])
                                           .astype('int') 
                                           <= cutoff))
        leaving_rate_df.at[programme, year] = (len(STUDENTS[selection_left_students]) 
                                                  / len(STUDENTS[selection_total_students]))
leaving_rate_df

In [ ]:
# calculate time to leaving
###############################################################################
'''
this isn't quite right, because the 3rd attempts are actually the following year
'''
###############################################################################
print('Time spent before leaving early: ')
for programme in programmes.tolist():
    df = (STUDENTS.loc[(STUDENTS['Year left prematurely'].notnull() 
                        & (STUDENTS['Programme'] == programme)), 'Year left prematurely']
                  .apply(lambda x: x[:2] + x[-2:])
                  .astype('int') 
        - STUDENTS.loc[(STUDENTS['Year started'].notnull() 
                        & (STUDENTS['Programme'] == programme)), 'Year started']
          .apply(lambda x: x[:4])
          .astype('int'))
    time = df.mean()
    print(programme, time)


In [ ]:
# calculate time to completion
print('Time to completion: ')
for programme in programmes.tolist():
    df = (STUDENTS.loc[(STUDENTS['Graduated year'].notnull() & (STUDENTS['Programme'] == programme)), 'Graduated year']
                  .apply(lambda x: x[:2] + x[-2:])
                  .astype('int') 
        - STUDENTS.loc[(STUDENTS['Year started'].notnull() & (STUDENTS['Programme'] == programme)), 'Year started']
          .apply(lambda x: x[:4])
          .astype('int'))
    time = df.mean()
    print(programme, time)

In [ ]:
for dfname, df in module_files.items():
    module_files[dfname]['Real Attempt Number(s)'] = np.empty((len(df),0)).tolist()

for course in ['CONAD','CONTRACT','LSM','TORT','EQUITY','LAND','CRIMINAL','EU','LT1','LT2']:
    for index, row in STUDENTS.iterrows():
        if any(row[f'{course} Attempts']):
            for record in row[f'{course} Attempts']:
                year = record[0]
                if year == '201819':
                    print(index + ' year 201819')
                    pass
                else:
                    attempts_that_year = record[-1]
                    module_files[f'{year}_{course}_assessment'].at[index,'Real Attempt Number(s)'] = attempts_that_year
                    # real attempt number is either empty list, integer, or list
                    # let's replace the empty lists with np.NaN

for dfname, df in module_files.items():
    for index, row in df.iterrows(): 
        if type(row['Real Attempt Number(s)']) == list:
            if not any(row['Real Attempt Number(s)']):
                module_files[dfname].at[index, 'Real Attempt Number(s)'] = np.NaN
                
dfname

In [ ]:
module_files[f'{year}_{course}_assessment'].to_excel("/Users/Kate/Desktop/Vicky project/data/example_moduledf.xls")

In [ ]:
year = '201314'
course = 'CONTRACT'
module_files[f'{year}_{course}_assessment'].sort_index()

In [ ]:
module_files[f'{year}_{course}_assessment'].columns

In [ ]:
assignment_numbers = {}
for dfname, df in module_files.items():
    assignment_numbers[dfname] = df.columns[-4].split()[1]
max_num_assignments = int(max(assignment_numbers.values())) #2
assignment_numbers;

In [ ]:
ATTEMPTS = {}
tuples = []
for index in STUDENTS.index:
    for course in  ['CONAD', 'CONTRACT', 'LSM', 'TORT', 'LAND', 'CRIMINAL', 'EQUITY', 'EU', 'LT1', 'LT2']:
        tuples.append((index,course))

index = pd.MultiIndex.from_tuples(tuples, names=['SPRcode', 'Module'])
        
for attempt in ['first', 'second', 'third', 'fourth', 'fifth']:
    ATTEMPTS[f'{attempt}'] = pd.DataFrame(index=index, columns = ['Student Programme','Student Final Status',
                                                    'Year of Attempt', 'Attempt Type', 'Module Mark', 
                                                     'Module Grade', 'Module Result'])

first_counter = 0
for dfname, df in module_files.items():
    year, module, tmp = dfname.split('_') 
    for index, row in df.iterrows():
        previous_year_name = year.split('_')[0][:2] + \
                        str(int(year.split('_')[0][2:4]) - 1) + \
                        str(int(year.split('_')[0][4:]) - 1) + '_' + module + '_' + 'assessment'
        next_year_name = year.split('_')[0][:2] + \
                        str(int(year.split('_')[0][2:4]) + 1) + \
                        str(int(year.split('_')[0][4:]) + 1) + '_' + module + '_' + 'assessment'
        try:
            previous_year_result = module_files[previous_year_name].at[index,'Result']
        except:
            previous_year_result = np.NaN 
        try:
            if index in module_files[next_year_name].index:
                if type(module_files[next_year_name].at[index,'Result']) != str:
                    # this year is a reassessment that was failed
                    second_reassessment_flag = True #worked for '0033fc70-2eb9-45f0-990e-d514bab75508'&'201112_CONAD_assessment'
                else:
                    second_reassessment_flag = False
            else:
                second_reassessment_flag = False             
        except:
            second_reassessment_flag = False # False -> took except route

        ####type(row['Result']) # str

        ####################################################################
        if (type(row['Result']) == str): 
            # if this isn't the second reassessment (which is recorded independently)
            primary_row = pd.Series()
            primary_row['Student Programme'] = STUDENTS.at[index,'Programme'] # PT
            primary_row['Student Final Status'] = STUDENTS.at[index, 'Current status'] # not progressing
            primary_row['Year of Attempt'] = year # 201718
            ####################################################################
             # if a reassessment
            ####################################################################
            if (row['Reassess Flag'] == True) | (row['DR Flag'] == True): # if the year includes a reassessment 
                if second_reassessment_flag == False:
                    reassessment_row = primary_row.copy() # otherwise will overwrite primary_row!!!
                    reassessment_row['Module Grade'] = row['Grade']
                    reassessment_row['Module Result'] = row['Result'] 
                    if any(df.columns == 'Mark'):
                        reassessment_row['Module Mark'] = row['Mark']
                    else:
                        reassessment_row['Module Mark'] = np.NaN
                    if row['DR Flag'] == True:
                        reassessment_row['Attempt Type'] = 'DR Reassessment'
                    else:
                        reassessment_row['Attempt Type'] = 'FR Reassessment'
                    try:
                        if type(row['Real Attempt Number(s)']) == list:
                            reassessment_attempt_num = row['Real Attempt Number(s)'][-1]
                        else:
                            first_attempt_that_year = row['Real Attempt Number(s)']
                        if reassessment_attempt_num == 1: # DR reassessment
                            ATTEMPTS['second'].loc[(index, module)] = reassessment_row
                        if reassessment_attempt_num == 2:
                            ATTEMPTS['second'].loc[(index, module)] = reassessment_row 
                        if reassessment_attempt_num == 3:
                            ATTEMPTS['third'].loc[(index,module)] = reassessment_row
                        if reassessment_attempt_num == 4:
                            ATTEMPTS['fourth'].loc[(index,module)] = reassessment_row
                        if reassessment_attempt_num == 5:
                            ATTEMPTS['fifth'].loc[(index,module)] = reassessment_row
                    except:
                        # if there is no attempt number (ie started before 2011), then we don't care to write it anyway
                        if type(STUDENTS.at[index,'Year started']) == str:
                            print(STUDENTS.at[index,'Year started'],'no attempt number', index, module, year)
                        pass
                elif second_reassessment_flag == True:
                    # will need to make a row each to put in first reassessment and second reassessment
                    first_reassessment_row = primary_row.copy() # do not overwrite primary_row
                    second_reassessment_row = primary_row.copy() # do not overwrite primary_row or first_assessment_row
                    first_reassessment_row['Module Grade'] = np.NaN
                    second_reassessment_row['Module Grade'] = row['Grade'] 
                    first_reassessment_row['Module Result'] = 'F calc?'
                    second_reassessment_row['Module Result'] = row['Result'] 
                    first_reassessment_row['Module Mark'] = np.NaN
                    if any(df.columns == 'Mark'):
                        second_reassessment_row['Module Mark'] = row['Mark']
                    else:
                        second_reassessment_row['Module Mark'] = np.NaN
                    if (row['DR Flag'] == True):
                        first_reassessment_row['Attempt Type'] = 'DR Reassessment'
                    else:
                        first_reassessment_row['Attempt Type'] = 'FR Reassessment'
                    second_reassessment_row['Attempt Type'] = 'Reassessment Following Year'
                    if type(row['Real Attempt Number(s)']) == list:
                        first_reassessment_attempt_num = row['Real Attempt Number(s)'][-1]
                    else:
                        first_reassessment_attempt_num = row['Real Attempt Number(s)']
                    if first_reassessment_attempt_num == 1:
                        print('error with attemptnum = 1 after reassessment',index, module, year)
                    if first_reassessment_attempt_num == 2:
                        ATTEMPTS['second'].loc[(index, module)] = first_reassessment_row
                        ATTEMPTS['third'].loc[(index, module)] = second_reassessment_row
                    if first_reassessment_attempt_num == 3:
                        ATTEMPTS['third'].loc[(index,module)] = first_reassessment_row
                        ATTEMPTS['fourth'].loc[(index, module)] = second_reassessment_row
                    if first_reassessment_attempt_num == 4:
                        ATTEMPTS['fourth'].loc[(index,module)] = first_reassessment_row
                        ATTEMPTS['fifth'].loc[(index, module)] = second_reassessment_row
                    if first_reassessment_attempt_num == 5:
                        ATTEMPTS['fifth'].loc[(index,module)] = first_reassessment_row
                    ####################################################################
                    # put primary rows in correct attempt dfs
                    ####################################################################
                if not ((pd.to_numeric(year,errors='coerce') > 201314) & (module =='LSM')): #needs testing
                    try:
                        if (int(df.columns[-4].split()[1]) == 2): # num assignments
                            primary_row['Module Mark'] = float((pd.to_numeric(row['Assessment 1 Weight']) 
                                                                * pd.to_numeric(row['Assessment 1 Mark']))
                                                               + (pd.to_numeric(row['Assessment 2 Weight']) 
                                                                * pd.to_numeric(row['Assessment 2 Mark'])))
                        else:
                            primary_row['Module Mark'] = row['Assessment 1 Mark']
                        if int(float(pd.to_numeric(primary_row['Module Mark']))) >= 40.0:
                            primary_row['Module Grade'] = 'P calc'
                            primary_row['Module Result'] = 'P calc'
                        else:
                            primary_row['Module Grade'] = 'F calc'
                            primary_row['Module Result'] = 'F calc'    
                    except:
                        print('error in calculating mark',index, module, year, row['Assessment 1 Mark'])
                        # this is a reassessment that is taking the full year
                        # perhaps is being recorded below
                else: # course is LSM and both assignments must be passed to pass
                    primary_row['Module Mark'] = np.NaN
                    if ((df.at[index,'Assessment 1 Grade'] in ['P','LP']) & (
                        df.at[index,'Assessment 2 Grade'] in ['P','LP'])):
                        primary_row['Module Grade'] = 'P calc'
                        primary_row['Module Result'] = 'P calc'
                    else:
                        primary_row['Module Grade'] = 'F calc'
                        primary_row['Module Result'] = 'F calc'
            ####################################################################
             # if no reassessment
            ####################################################################
            else: # no reassessment that year
                primary_row['Module Grade'] = row['Grade']
                primary_row['Module Result'] = row['Result'] 
                if any(df.columns == 'Mark'):
                    primary_row['Module Mark'] = row['Mark']
                else:
                    primary_row['Module Mark'] = np.NaN

            ####################################################################
             # store the first primary row
            ####################################################################
            if type(row['Real Attempt Number(s)']) == list:
                first_attempt_that_year = row['Real Attempt Number(s)'][0]
            else:
                first_attempt_that_year = row['Real Attempt Number(s)']
            if (first_attempt_that_year == 1): # initial assessment
                first_counter +=1
                if previous_year_result != 'D':
                    primary_row['Attempt Type'] = 'Initial Assessment'
                    ATTEMPTS['first'].loc[(index, module)] = primary_row
                else:
                    primary_row['Attempt Type'] = 'DR Retake'
                    ATTEMPTS['second'].loc[(index, module)] = primary_row
            else:# make retake
                retake_attempt_num = first_attempt_that_year
                primary_row['Attempt Type'] = 'Retake'
                if retake_attempt_num == 2:
                    if previous_year_result != 'D':
                        ATTEMPTS['second'].loc[(index, module)] = primary_row
                    else:
                        primary_row['Attempt Type'] = 'DR Retake'
                        ATTEMPTS['third'].loc[(index, module)] = primary_row
                elif retake_attempt_num == 3:
                    if previous_year_result != 'D':
                        ATTEMPTS['third'].loc[(index,module)] = primary_row
                    else:
                        primary_row['Attempt Type'] = 'DR Retake'
                        ATTEMPTS['fourth'].loc[(index, module)] = primary_row     
                elif retake_attempt_num == 4:
                    if previous_year_result != 'D':
                        ATTEMPTS['fourth'].loc[(index,module)] = primary_row
                    else:
                        primary_row['Attempt Type'] = 'DR Retake'
                        ATTEMPTS['fifth'].loc[(index, module)] = primary_row
                elif retake_attempt_num == 5:
                    ATTEMPTS['fifth'].loc[(index,module)] = primary_row 

In [ ]:
first_counter

In [ ]:
for attempt in ['first', 'second', 'third', 'fourth', 'fifth']:
    ATTEMPTS[f'{attempt}'].dropna(how='all', inplace = True) 
    ATTEMPTS[f'{attempt}']['Module Mark'] = pd.to_numeric(ATTEMPTS[f'{attempt}']['Module Mark'])

In [ ]:
ATTEMPTS['first']['Attempt Type'].value_counts(dropna=False)

In [ ]:
first_attempts = ATTEMPTS['first'].copy()
first_attempts.reset_index(inplace=True)
first_attempts

In [ ]:
ATTEMPTS['second']['Attempt Type'].value_counts(dropna=False)

In [ ]:
second_attempts = ATTEMPTS['second'].copy()
second_attempts

In [ ]:
first_attempts.loc[first_attempts['Module Result'] == 'P calc']

In [ ]:
tuples = first_attempts.loc[first_attempts['Module Result'] == 'P calc', ['SPRcode', 'Module']].apply(lambda x: (x['SPRcode'], x['Module']), axis=1).tolist()

tuples


In [ ]:
ATTEMPTS['second'].loc[tuples, :]

In [ ]:
num_attempt_2_is_retake = len(second_attempts[second_attempts['Attempt Type'].str.contains('Retake')]) 
num_pass_attempt_2_when_retake = len(second_attempts[(second_attempts['Attempt Type'].str.contains('Retake') 
                                                & ((second_attempts['Module Result'] == 'P') 
                                                   | (second_attempts['Module Result'] == 'P calc')))])
prop_pass_attempt_2_when_retake = num_pass_attempt_2_when_retake / num_attempt_2_is_retake
prop_pass_attempt_2_when_retake

In [ ]:
num_attempt_2_is_reassess = len(second_attempts[second_attempts['Attempt Type'].str.contains('FR Reassessment')]) # 1079
num_pass_attempt_2_when_reassess = len(second_attempts[(second_attempts['Attempt Type'].str.contains('FR Reassessment')
                                                      & ((second_attempts['Module Result'] == 'P') 
                                                       | (second_attempts['Module Result'] == 'P calc')))])
prop_pass_attempt_2_when_reassess = num_pass_attempt_2_when_reassess / num_attempt_2_is_reassess
prop_pass_attempt_2_when_reassess

In [ ]:
# Make a df with each attempt pass rate as cell value, columns = years, rows = modules

############### 
'''May want to add a condition that must have failed first attempt'''
############
second_attempts = ATTEMPTS['second'].copy()
second_attempts.reset_index(inplace=True)

modules = ['LSM', 'CONTRACT', 'CONAD']#, 'TORT', 'CRIMINAL', 'LAND', 'EQUITY', 'EU', 'LT1', 'LT2']
years = sorted(second_attempts['Year of Attempt'].value_counts().index.tolist())

second_attempts_reassess_pt_yrs = pd.DataFrame()
second_attempts_reassess_ft_yrs = pd.DataFrame()
second_attempts_retake_pt_yrs = pd.DataFrame()
second_attempts_retake_ft_yrs = pd.DataFrame()


for year in years:
    for module in modules:
        pt_reassess_selection_total = ((second_attempts['Student Programme'] == 'PT') 
                           & second_attempts['Attempt Type'].str.contains('FR Reassessment') 
                           & (second_attempts['Module'] == module)
                           & (second_attempts['Year of Attempt'] == year))  
        pt_reassess_selection_passed = (pt_reassess_selection_total & ((second_attempts['Module Result'] == 'P') 
                                              | (second_attempts['Module Result'] == 'P calc'))) 
        pt_retake_selection_total = ((second_attempts['Student Programme'] == 'PT') 
                           & second_attempts['Attempt Type'].str.contains('Retake') 
                           & (second_attempts['Module'] == module)
                           & (second_attempts['Year of Attempt'] == year)) 
        pt_retake_selection_passed = (pt_retake_selection_total & ((second_attempts['Module Result'] == 'P') 
                                              | (second_attempts['Module Result'] == 'P calc')))
        
        ft_reassess_selection_total = ((second_attempts['Student Programme'] == 'FT') 
                           & second_attempts['Attempt Type'].str.contains('FR Reassessment') 
                           & (second_attempts['Module'] == module)
                           & (second_attempts['Year of Attempt'] == year))  
        ft_reassess_selection_passed = (ft_reassess_selection_total & ((second_attempts['Module Result'] == 'P') 
                                              | (second_attempts['Module Result'] == 'P calc'))) 
        ft_retake_selection_total = ((second_attempts['Student Programme'] == 'FT') 
                           & second_attempts['Attempt Type'].str.contains('Retake') 
                           & (second_attempts['Module'] == module)
                           & (second_attempts['Year of Attempt'] == year)) 
        ft_retake_selection_passed = (ft_retake_selection_total & ((second_attempts['Module Result'] == 'P') 
                                              | (second_attempts['Module Result'] == 'P calc')))
        
        if (int(len(second_attempts[pt_reassess_selection_total])) != 0):
            second_attempts_reassess_pt_yrs.at[module, year] = \
            len(second_attempts[pt_reassess_selection_passed])/len(second_attempts[pt_reassess_selection_total]) 
        
        if (int(len(second_attempts[pt_retake_selection_total])) != 0):
            second_attempts_retake_pt_yrs.at[module, year] = \
            len(second_attempts[pt_retake_selection_passed])/len(second_attempts[pt_retake_selection_total])
            
        if (int(len(second_attempts[ft_reassess_selection_total])) != 0):
            second_attempts_reassess_ft_yrs.at[module, year] = \
            len(second_attempts[ft_reassess_selection_passed])/len(second_attempts[ft_reassess_selection_total]) 
        
        if (int(len(second_attempts[ft_retake_selection_total])) != 0):
            second_attempts_retake_ft_yrs.at[module, year] = \
            len(second_attempts[ft_retake_selection_passed])/len(second_attempts[ft_retake_selection_total])            

In [ ]:
second_attempts_reassess_pt_yrs.mean()

In [ ]:
second_attempts_retake_pt_yrs.mean()

In [ ]:
second_attempts_reassess_ft_yrs

In [ ]:
second_attempts_retake_ft_yrs

In [ ]:
# Make a df with each attempt pass rate as cell value, columns = years, rows = modules

modules = ['LSM', 'CONTRACT', 'CONAD']#, 'TORT', 'CRIMINAL', 'LAND', 'EQUITY', 'EU', 'LT1', 'LT2']
years = sorted(first_attempts['Year of Attempt'].value_counts().index.tolist())

first_attempts_pt_yrs = pd.DataFrame()
first_attempts_ft_yrs = pd.DataFrame()

pt_selection_total = pd.DataFrame()
ft_selection_total = pd.DataFrame()

for year in years:
    for module in modules:
        pt_selection_total[year] = ((first_attempts['Student Programme'] == 'PT')  
                           & (first_attempts['Module'] == module)
                           & (first_attempts['Year of Attempt'] == year))  
        pt_selection_passed = (pt_selection_total[year] & ((first_attempts['Module Result'] == 'P') 
                                              | (first_attempts['Module Result'] == 'P calc')))    
        ft_selection_total[year] = ((first_attempts['Student Programme'] == 'FT') 
                           & (first_attempts['Module'] == module)
                           & (first_attempts['Year of Attempt'] == year))  
        ft_selection_passed = (ft_selection_total[year] & ((first_attempts['Module Result'] == 'P') 
                                              | (first_attempts['Module Result'] == 'P calc'))) 
           
        if (int(len(first_attempts[pt_selection_total[year]])) != 0):
            first_attempts_pt_yrs.at[module, year] = \
            len(first_attempts[pt_selection_passed])/len(first_attempts[pt_selection_total[year]]) 
            
        if (int(len(first_attempts[ft_selection_total[year]])) != 0):
            first_attempts_ft_yrs.at[module, year] = \
            len(first_attempts[ft_selection_passed])/len(first_attempts[ft_selection_total[year]])            

In [ ]:
first_attempts_pt_yrs.mean()

In [ ]:
first_attempts_ft_yrs.mean()

In [ ]:
first_attempts[pt_selection_total].sum(axis=0)

In [ ]:
first_attempts_ft_yrs.iloc[0:4].mean()

In [ ]:
first_attempts_ft_yrs.mean(axis=1)

In [ ]:
first_attempts_pt_yrs.iloc[0:3].mean()

In [ ]:
first_attempts_pt_yrs.mean(axis=1)

In [ ]:
third_attempts = ATTEMPTS['third']

In [ ]:
third_attempts['Attempt Type'].value_counts(dropna=False)

In [ ]:
ATTEMPTS['third']['Module Result'].value_counts(dropna=False)

In [ ]:
ATTEMPTS['fourth']

In [ ]:
ATTEMPTS['fifth']

In [ ]:
third_attempts[third_attempts['Attempt Type'].isnull()]